In [1]:
import numpy as np 
import pandas as pd 

In [2]:
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001

def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character
    print('%s' % (txt, ), end='')


def initialize_parameters(n_a, n_x, n_y):
    Wax = np.random.randn(n_a, n_x)*0.01  # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01  # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01  # hidden to output
    ba = np.zeros((n_a, 1))  # hidden bias
    by = np.zeros((n_y, 1))  # output bias

    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba, "by": by}

    return parameters


def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['ba'] += -lr * gradients['db']
    parameters['by'] += -lr * gradients['dby']
    
    return parameters


In [3]:
def rnn_cell_forward(xt, a_prev, parameters):
    
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]

    a_next = np.tanh(np.dot(Wax, xt) + np.dot(Waa, a_prev) + ba)
    yt_pred = softmax(np.dot(Wya, a_next) + by)

    cache = (a_next, a_prev, xt, parameters)

    return a_next, yt_pred, cache

def rnn_forward(X, Y, a0, parameters, vocab_size=27):
    x, a, y_hat = {}, {}, {}
    a[-1] = np.copy(a0) #a[-1] = a_prev, initially list of zeroes of dim (n_a, 1) 
    loss = 0
    for t in range(len(X)): # Word h X...
        x[t] = np.zeros((vocab_size, 1))
        if (X[t] != None):
            x[t][X[t]] = 1
        a[t], y_hat[t], _ = rnn_cell_forward(x[t], a[t-1],parameters) #shimple, initially a[-1] is 0.. So a[1] and y_hat[1] will only depend on Wax and first letter in word
#         if t < 10:     
#             print(str(t) + 'A[T] IS EQUAL TO ' + str(a[t]) + 'Y[T IS EQUAL TO]' + str(y_hat[t])) # isko hata dena
        loss -= np.log(y_hat[t][Y[t], 0])
    cache = (y_hat, a, x)
    return loss, cache

In [4]:
def rnn_cell_backward(dy, gradients, parameters, x, a, a_prev):
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) +         gradients['da_next']  # backprop into h
    daraw = (1 - a * a) * da  # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients


def rnn_backward(X, Y, parameters, cache):
    gradients = {}
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(ba), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])

    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_cell_backward(
            dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

In [5]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


def clip(gradients, maxValue):
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


def sample(parameters, char_to_ix, seed):
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]    
    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))
    indices = []
    
    idx = -1 
    
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)
            
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        indices.append(idx)
        
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        a_prev = a
        
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices


def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    gradients = clip(gradients, 5)
    
    parameters = update_parameters(parameters, gradients, learning_rate)
        
    return loss, gradients, a[len(X)-1]


In [11]:
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    
    n_x, n_y = vocab_size, vocab_size
    
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    loss = get_initial_loss(vocab_size, dino_names)
    
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    np.random.seed(0)
    np.random.shuffle(examples)
    
    a_prev = np.zeros((n_a, 1))
    
    for j in range(num_iterations):
        
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        #print(Y) #dekhe to zara X, Y hai kya 
        
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters) #parameters ek dictionary h, ie Weights are upgraded each iteration 
        
        loss = smooth(loss, curr_loss) # what ij this

        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            print('A_PREV H: ' + str(a_prev))
            seed = 0
            for name in range(dino_names):
                print(a_prev)
                sampled_indices = sample(parameters, char_to_ix, seed)
                txt = ''.join(ix_to_char[ix] for ix in sampled_indices)
                txt = txt[0].upper() + txt[1:]  # capitalize first character
                print('%s' % (txt, ), end='')
                
                seed += 1  
      
            print('\n')
        
    return parameters


In [7]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }

#model(data, ix_to_char, char_to_ix)

There are 19909 total characters and 27 unique characters in your data.


In [8]:
print(char_to_ix)
print(ix_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [9]:
model(data, ix_to_char, char_to_ix) #keeping a_n 50

Iteration: 0, Loss: 23.087340

Co
Eyhqgyshubjerbjgmis
Qfguyhvbqtdkxb
Okuxhjmozrjgycsxqkeisqywvxgijorg
Jhywwqinubuavuapmdctozzzcyflteoebci
Kqjbagdzduoomziuojypckurxgtloyzwiwqk
Tmutuhunlr


Iteration: 2000, Loss: 27.905623

Amychamgvriatolycrhesiurntoranbsausuwtaantotaaurus
Honpolgerotosaurus
Esakanrosillostfrsaurimtanelocsaurus
Egngxypovtoncclytcatonosaunus
Paeileloyouros
Konusaurus
Aurus


Iteration: 4000, Loss: 25.800830

Lusaurus
Ythosaurus
Onsaurus
Alavotis
Giphrya
Aceronetisaurus
Nonosaurus


Iteration: 6000, Loss: 24.520959

Steoptetterntropholeonsauruys
Brantssibhimalosauruh
Eroshustsinazdos
Dernotes
Huyeodos
Lenospcelenosaususaura
Gipeosauruscheralherostestrosaurus


Iteration: 8000, Loss: 24.019045

Crojhusaurus
Tameitorydularaprosaurus
Eluimas
Eltesoults
Haindkhan
Ntraptoros



Iteration: 10000, Loss: 23.807775

Cereokalttesaurus
Storopholena
Soptatenathria
Rtoromicur
Hiaosaurus
Naryrgatotopandonpsaurus
Xahosaurus


Iteration: 12000, Loss: 23.163348

Heugnosaurus
Inttosaurus
C

{'Wax': array([[-2.52921050e-03, -2.94196457e-01,  1.58040679e-01, ...,
          9.93027565e-02,  9.84744550e-01, -1.01673304e+00],
        [ 9.89461215e-03,  1.16474086e+00,  7.15050233e-01, ...,
          4.62470213e-02, -1.17782133e+00,  5.49753746e-02],
        [ 1.33345578e-03,  1.72628835e-02, -4.57335709e-01, ...,
          6.20223045e-02,  3.06365545e-01, -2.39961289e-01],
        ...,
        [ 7.01672654e-03, -2.18703705e+00,  1.20637681e+00, ...,
          5.49807258e-01,  4.37126272e-01,  3.34793786e-01],
        [-1.33419646e-03, -1.22884654e+00, -4.29996225e-01, ...,
          2.05799326e-01, -1.24212781e+00,  3.91213655e-01],
        [-4.28496805e-03, -1.05013243e+00, -2.88944243e-01, ...,
         -3.74141933e-01,  7.87342869e-01,  3.17608247e-02]]),
 'Waa': array([[-0.27043363, -0.20864296,  0.04255665, ..., -0.25463049,
          0.13164283,  0.04288117],
        [-0.46476539,  0.62612212, -0.19667409, ...,  0.54411396,
          0.17509278,  0.40057535],
        [-0

In [11]:
model(data, ix_to_char, char_to_ix) #keeping a_n 45

Iteration: 0, Loss: 23.087332

Co
Eyhqgyshubjerbjgmis
Qfguyhvbqtdkxb
Okuxhjmozrjgycsxqkeisqywvxgijorg
Jhywwqinubuavuapmdctozzzcyflteoebci
Kqjbagdzduoomziuojypckurxgtloyzwiwqk
Tmutuhunlr


Iteration: 2000, Loss: 27.878748

Alycgangvriatolzathiseuriseugolrosttypa
Etrybaurutisaurus
Lpururtirelhrodansaurus
Rurusaus
Aukurus
Tinnonamcniutorytongblytcatomosaupnadpacengdoypurh
Genoptrosaurusaurus


Iteration: 4000, Loss: 25.827818

Trisaurus
Ishondos
Ngthosaurus
Votos
Elonstalacerophus
Lypalisaurus
Ntiros


Iteration: 6000, Loss: 24.648286

Niteus
Lostoras
Celroptox
Ibrantros
Limalosaurun
Erositovses
Zarriianltreus


Iteration: 8000, Loss: 24.173113

Uyhodosaurus
Ualdenisaurus
Eubalosaurus
Ittras
Delnorur
Shinionhodesaurus
Danosechoenmives


Iteration: 10000, Loss: 23.884884

Asaurosaurus
Amugnateodolusaleranacheicanatos
Mrerosalheokonacotataus
Avotonjiosaurus
Pixasaurus
Ocrotoromelus
Doanosaurus


Iteration: 12000, Loss: 23.213235

Apxoncisus
Anesaurus
Sapuelosaurus
Ditcerusaurteratrosaurus
A

{'Wax': array([[-2.26122470e-03,  4.98713946e-01, -6.14402741e-01, ...,
          1.77806155e+00, -4.37900988e-01,  1.77759152e-01],
        [-6.65041618e-03,  2.52121860e+00, -8.09582691e-01, ...,
          9.44799709e-01,  1.58927303e+00, -8.52675241e-01],
        [-1.19325783e-02,  1.17723313e+00, -6.34327597e-01, ...,
          1.08540819e+00,  6.88861314e-01, -2.42580788e-01],
        ...,
        [ 2.09806931e-03, -3.90406863e-01, -9.09350079e-01, ...,
         -6.17324521e-01,  2.17839865e-01, -1.78733956e-01],
        [-5.48137468e-03,  6.10089868e-02, -1.22142929e-01, ...,
          4.62427918e-01,  4.05596426e-01, -1.72858173e-01],
        [-3.88642973e-03, -3.40670174e+00, -7.20553030e-02, ...,
         -1.76929581e-01, -6.84863304e-01, -4.47684770e-02]]),
 'Waa': array([[-0.69340054,  0.4785507 ,  0.07435874, ...,  0.13545609,
          0.34017052,  0.2053262 ],
        [-0.70258247, -0.19175264, -0.32125747, ...,  0.10375462,
          0.08151028, -0.64779973],
        [ 0

In [10]:
with open("dinos.txt") as f:
        examples = f.readlines()
examples = [x.lower().strip() for x in examples]
    
print(examples)

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor', 'achillesaurus', 'achillobator', 'acristavus', 'acrocanthosaurus', 'acrotholus', 'actiosaurus', 'adamantisaurus', 'adasaurus', 'adelolophus', 'adeopapposaurus', 'aegyptosaurus', 'aeolosaurus', 'aepisaurus', 'aepyornithomimus', 'aerosteon', 'aetonyxafromimus', 'afrovenator', 'agathaumas', 'aggiosaurus', 'agilisaurus', 'agnosphitys', 'agrosaurus', 'agujaceratops', 'agustinia', 'ahshislepelta', 'airakoraptor', 'ajancingenia', 'ajkaceratops', 'alamosaurus', 'alaskacephale', 'albalophosaurus', 'albertaceratops', 'albertadromeus', 'albertavenator', 'albertonykus', 'albertosaurus', 'albinykus', 'albisaurus', 'alcovasaurus', 'alectrosaurus', 'aletopelta', 'algoasaurus', 'alioramus', 'aliwalia', 'allosaurus', 'almas', 'alnashetri', 'alocodon', 'altirhinus', 'altispinax', 'alvarezsaurus', 'alwalkeria', 'alxasaurus', 'amargasaurus', 'amarga

In [ ]:
model(data, ix_to_char, char_to_ix) # n_iter ko vapas 35k aur print(X,Y) hatana h

In [11]:
len(examples)

1536

In [14]:
print([None] + [char_to_ix[ch] for ch in examples[15]])
print([None] + [char_to_ix[ch] for ch in examples[15]][1:] + [char_to_ix["\n"]])

[None, 1, 3, 20, 9, 15, 19, 1, 21, 18, 21, 19]
[None, 3, 20, 9, 15, 19, 1, 21, 18, 21, 19, 0]


In [15]:
examples[15]

'actiosaurus'

In [17]:
print(len(data))

19909


In [19]:
print(np.zeros((50,1)))

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


In [12]:
model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.087338

A_PREV H: [[-2.91284836e-03]
 [-8.29234484e-03]
 [ 1.73792213e-02]
 [-4.21515917e-03]
 [ 1.83474528e-03]
 [ 1.97950534e-03]
 [ 4.35562079e-03]
 [-7.23729158e-04]
 [ 6.27008329e-03]
 [ 9.67745568e-03]
 [ 5.50352331e-03]
 [ 1.96420919e-02]
 [-7.74860746e-03]
 [-1.42003823e-02]
 [-1.40785085e-02]
 [ 1.06486673e-03]
 [ 7.58203734e-03]
 [ 6.52778882e-03]
 [ 9.07520307e-03]
 [ 2.13387865e-03]
 [-1.00166261e-03]
 [ 2.21537696e-03]
 [ 1.43770543e-03]
 [-8.73884575e-03]
 [ 8.80737080e-03]
 [-6.17956166e-03]
 [ 4.97815882e-03]
 [-1.79320721e-02]
 [-9.04271988e-03]
 [ 3.69261059e-04]
 [ 5.22627434e-03]
 [-7.91599140e-03]
 [-1.88776741e-02]
 [ 3.05227621e-04]
 [-2.90654542e-03]
 [ 1.57767485e-02]
 [-3.21654165e-03]
 [ 4.44298007e-03]
 [-2.57972537e-03]
 [-5.32280491e-03]
 [ 1.97277052e-02]
 [ 1.34694759e-02]
 [ 4.64407232e-03]
 [ 2.08864532e-03]
 [-1.28742420e-05]
 [ 5.50176391e-03]
 [ 5.45328764e-03]
 [-1.73666294e-02]
 [ 1.14885611e-02]
 [ 7.54520587e-03]]
[[-2.912

Iteration: 4000, Loss: 25.812775

A_PREV H: [[ 0.21295347]
 [ 0.24254603]
 [ 0.93627012]
 [-0.35877987]
 [-0.19414502]
 [ 0.89146787]
 [-0.57962974]
 [ 0.51589615]
 [-0.43878378]
 [ 0.84384698]
 [ 0.6540942 ]
 [ 0.74194687]
 [ 0.51327943]
 [-0.70077015]
 [ 0.56125449]
 [ 0.20170346]
 [ 0.30393206]
 [-0.2632402 ]
 [ 0.40587516]
 [ 0.44574423]
 [-0.1893503 ]
 [-0.33479709]
 [ 0.27678586]
 [-0.62546124]
 [-0.21995482]
 [ 0.17107933]
 [-0.40166136]
 [-0.16360956]
 [-0.08228442]
 [-0.06419998]
 [-0.13804776]
 [ 0.68892925]
 [-0.41333573]
 [ 0.00510321]
 [ 0.40015915]
 [ 0.38931149]
 [-0.18981683]
 [ 0.15637392]
 [ 0.15469961]
 [-0.43677227]
 [ 0.95921938]
 [-0.35013739]
 [ 0.18621922]
 [ 0.97744855]
 [ 0.59669741]
 [ 0.73782623]
 [-0.06325065]
 [-0.58668783]
 [ 0.75631009]
 [-0.31718323]]
[[ 0.21295347]
 [ 0.24254603]
 [ 0.93627012]
 [-0.35877987]
 [-0.19414502]
 [ 0.89146787]
 [-0.57962974]
 [ 0.51589615]
 [-0.43878378]
 [ 0.84384698]
 [ 0.6540942 ]
 [ 0.74194687]
 [ 0.51327943]
 [-0.70077

Iteration: 8000, Loss: 24.114794

A_PREV H: [[ 0.31975147]
 [-0.88321712]
 [-0.98396417]
 [-0.60066191]
 [-0.99844189]
 [ 0.75995697]
 [ 0.22814087]
 [ 0.74165537]
 [-0.46450958]
 [-0.71604195]
 [-0.59267449]
 [-0.04465117]
 [ 0.16609872]
 [-0.73387716]
 [-0.99669475]
 [ 0.12071078]
 [ 0.93388005]
 [ 0.58598315]
 [-0.72602974]
 [ 0.4955286 ]
 [ 0.78618918]
 [-0.14221363]
 [-0.11030813]
 [-0.35755432]
 [ 0.00542058]
 [ 0.1592793 ]
 [-0.22176691]
 [ 0.95328398]
 [ 0.152553  ]
 [ 0.20379288]
 [-0.289433  ]
 [-0.76188237]
 [ 0.19903015]
 [ 0.56625444]
 [-0.52267574]
 [-0.34782597]
 [ 0.49590298]
 [-0.283512  ]
 [-0.09692886]
 [-0.51883383]
 [-0.95541018]
 [-0.95279755]
 [-0.2424715 ]
 [-0.41195714]
 [ 0.62333728]
 [ 0.19091602]
 [-0.26748034]
 [-0.33056417]
 [ 0.87915069]
 [-0.25950534]]
[[ 0.31975147]
 [-0.88321712]
 [-0.98396417]
 [-0.60066191]
 [-0.99844189]
 [ 0.75995697]
 [ 0.22814087]
 [ 0.74165537]
 [-0.46450958]
 [-0.71604195]
 [-0.59267449]
 [-0.04465117]
 [ 0.16609872]
 [-0.73387

Iteration: 12000, Loss: 23.166064

A_PREV H: [[ 0.21678101]
 [-0.91536678]
 [ 0.96930973]
 [-0.00932691]
 [-0.89688066]
 [ 0.8249953 ]
 [-0.69782921]
 [-0.90242983]
 [-0.91805911]
 [-0.92988183]
 [-0.1273617 ]
 [ 0.68602293]
 [ 0.92422705]
 [-0.97129201]
 [ 0.72348698]
 [ 0.12524251]
 [ 0.96606591]
 [ 0.13296091]
 [ 0.34515475]
 [ 0.88027236]
 [ 0.0118998 ]
 [-0.48523845]
 [ 0.92408812]
 [-0.88938592]
 [ 0.46300315]
 [ 0.26182512]
 [-0.66623246]
 [ 0.71300535]
 [ 0.98649273]
 [-0.23789674]
 [ 0.63631787]
 [ 0.24167109]
 [-0.61401313]
 [ 0.63852024]
 [ 0.2108237 ]
 [ 0.40071671]
 [ 0.57915071]
 [-0.47583529]
 [ 0.82007664]
 [-0.36116311]
 [ 0.93603594]
 [-0.99800019]
 [ 0.12480678]
 [ 0.97245164]
 [ 0.87996618]
 [ 0.93918758]
 [-0.56216093]
 [-0.88954726]
 [ 0.88815198]
 [ 0.48001036]]
[[ 0.21678101]
 [-0.91536678]
 [ 0.96930973]
 [-0.00932691]
 [-0.89688066]
 [ 0.8249953 ]
 [-0.69782921]
 [-0.90242983]
 [-0.91805911]
 [-0.92988183]
 [-0.1273617 ]
 [ 0.68602293]
 [ 0.92422705]
 [-0.9712

KeyboardInterrupt: 